<a href="https://colab.research.google.com/github/hui09241/10-fold/blob/master/10_fold_0923_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydrive
!pip install scikit-learn==0.20.0
!pip show scikit-learn # to see which version and where scikit-learn is installed
!pip freeze # to see all packages installed in the active virtualenv
"import sklearn; sklearn.show_versions()"

Name: scikit-learn
Version: 0.20.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, numpy
Required-by: yellowbrick, umap-learn, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute
absl-py==0.10.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
argon2-cffi==20.1.0
asgiref==3.2.10
astor==0.8.1
astropy==4.0.1.post1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.6
atomicwrites==1.4.0
attrs==20.2.0
audioread==2.1.8
autograd==1.3
Babel==2.8.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.2.0
blis==0.4.1
bokeh==2.1.1
boto==2.49.0
boto3==1.14.63
botocore==1.17.63
Bottleneck==1.3.2
branca==0.4.1
bs4==0.0.1
CacheControl==0.12.6
cachetools==4.1.1
catalogue==1.0.0
certifi==2020.6.20
cffi==1.14.2
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3

'import sklearn; sklearn.show_versions()'

In [ ]:
import pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization #這行milk02.ipynb
from keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error  #02.ipynb

import pandas as pd
from pydrive.auth import GoogleAuth #雲端部分#載入資料用
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

#把要one hot的類別轉換成數字                           #從這以下milk02.ipynb
from sklearn.preprocessing import LabelEncoder

#把要的類別轉換成one hot
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.optimizers import Adam

from sklearn.model_selection import KFold

In [ ]:
##載入new_train
auth.authenticate_user()                  
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '11jLvYsp8RkDqZioXL5j2Qitx7C2lwM79'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('our_new_train.csv')
new_train = pd.read_csv('our_new_train.csv', sep=',')
#new_train

In [ ]:
##載入new_test
auth.authenticate_user()                  
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1pQC3R9smKnmsqQEU3BUoWVmEk7YRK5Io'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('newtest17.csv')
new_test = pd.read_csv('newtest17.csv', sep=',')
#print(new_test.loc[[48]])
#new_test

In [ ]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1TgijKpsq6haP2C6yHsA7_bVtf5Ra6Cps'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('spec01.csv')
spec = pd.read_csv('spec01.csv',sep=',')
#print(spec)

In [ ]:
for i in range(len(spec['7'])):
    if(str(spec['7'][i])==str("A")):
          spec['7'][i]=1
for i in range(len(spec['7'])):
    if(str(spec['7'][i])==str('B')):
          spec['7'][i]=2
for i in range(len(spec['7'])):
    if(str(spec['7'][i])==str('C')):
          spec['7'][i]=3
#spec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#測試spec
#spec.loc[[4290]]

In [ ]:
new_train=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','DadID','BabyNum','FeedDay','Milk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
#new_train

In [ ]:
new_test=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','DadID','BabyNum','FeedDay','Milk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
#new_test

In [ ]:
new_train['Milk']

0        20.0
1        43.0
2        24.0
3        18.2
4        10.0
         ... 
30009    24.0
30010    20.4
30011    19.0
30012     NaN
30013    25.5
Name: Milk, Length: 30014, dtype: float64

In [ ]:
import math
bool = False
for i in range(len(new_train['Milk'])):
    bool = False
    bool=math.isnan(new_train['Milk'][i])
    if(bool):
       new_train['Milk'][i]=0
#print(new_train['Milk'].loc[[15]])
#print(new_train['Milk'].loc[[16]])
#new_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
import math
bool = False
for i in range(len(new_test['Milk'])):
    bool = False
    bool=math.isnan(new_test['Milk'][i])
    if(bool):
       new_test['Milk'][i]=0
#print(new_train['Milk'].loc[[15]])
#print(new_test['Milk'].loc[[9]])
#new_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
#train有不需要判斷的編號就去掉
for i in range(len(new_test['CowID'])):
    if len(new_train.index[new_train['CowID'] == new_test['CowID'][i]]) == 0:
        new_train=new_train.drop([i])
new_train.reset_index(inplace=True)
#new_train

In [ ]:
#train牛乳產量是0，則丟棄不用
for i in range(len(new_train['Milk'])): 
    if new_train['Milk'][i] == 0 :
        new_train = new_train.drop([i])
new_train.reset_index(inplace=True)
#new_train

In [ ]:
#test牛乳產量是0，則丟棄不用(未來測試需要)
for i in range(len(new_test['Milk'])): 
    if new_test['Milk'][i] == 0 :
        new_test = new_test.drop([i])
new_test.reset_index(inplace=True)
print(new_test['Milk'].loc[[9]])
new_test

9    11.6
Name: Milk, dtype: float64


,index,ID,firm,CowID,MomID,DadID,BabyNum,FeedDay,Milk,Sampling date,Age,BreedNum,Temperature,T Max,RH,RHMin,Precp,THI,cycle,tcalving number,disease,Sampling D Year,Sampling D Month
0,0,18072,2,122630,98111436.0,007HO08361,3,183,40.0,2017/1/19,70,2,20.8,25.5,85.0,69.0,0.0,20.236278,NaN,NaN,NaN,NaN,NaN
1,1,15357,2,2122398,98111423.0,7H7536,3,71,28.0,2017/7/18,50,0,NaN,35.2,82.0,69.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37069,3,99183211,94172172.0,1H8328,1,365,12.0,2013/6/25,38,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,17574,2,122607,98115809.0,029HO11295,4,177,35.0,2017/3/20,73,1,24.3,29.1,81.0,65.0,0.0,23.221440,NaN,NaN,NaN,NaN,NaN
4,4,3256,1,1051850,96040547.0,7H9502,1,42,28.8,2015/2/5,28,0,10.4,11.1,71.0,55.0,3.5,10.980262,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,7496,11987,2,1123347,99116307.0,7H8361,3,18,39.0,2015/8/20,42,0,31.0,34.9,82.0,61.0,0.0,29.323086,NaN,NaN,NaN,NaN,NaN
6679,7497,23863,2,99127667,NaN,200H1715,2,508,10.0,2014/7/21,52,4,31.5,34.8,72.0,54.0,0.0,28.814617,NaN,NaN,NaN,NaN,NaN
6680,7498,24192,2,97124147,NaN,7H6155,3,158,30.6,2013/5/27,62,1,29.3,34.4,75.0,51.0,0.0,27.200232,NaN,NaN,NaN,NaN,NaN
6681,7500,14623,2,2122364,95110600.0,29H10124,1,9,20.0,2014/12/25,22,0,16.9,19.5,80.0,60.0,5.5,16.578618,NaN,NaN,NaN,NaN,NaN


In [ ]:
#處理new_train的
#父親精液編號 
new_train['DadID']=new_train['DadID'].fillna("NoData").astype("str")

#將MomID中沒有編號的打為NoData 
new_train['MomID']=new_train['MomID'].fillna("NoData").astype("str")

avg_Temperature_tr=new_train['Temperature'].mean() #將平均溫度Nan部分以算術平均數之平均溫度取代
new_train['Temperature']= new_train['Temperature'].fillna(avg_Temperature_tr)


avg_T_Max_tr=new_train['T Max'].mean()          #將最高溫度Nan部分以算術平均數之最高溫度取代
new_train['T Max']= new_train['T Max'].fillna(avg_T_Max_tr)


avg_RH_tr=new_train['RH'].mean()                  #將相對濕度Nan部分以算術平均數之相對濕度取代
new_train['RH']= new_train['RH'].fillna(avg_RH_tr)


avg_RH_Min_tr=new_train['RHMin'].mean()         #將最小相對濕度Nan部分以算術平均數之最小相對濕度取代
new_train['RHMin']= new_train['RHMin'].fillna(avg_RH_Min_tr)


avg_Precp_tr=new_train['Precp'].mean()          #將降水量Nan部分以算術平均數之降水量取代
new_train['Precp']= new_train['Precp'].fillna(avg_Precp_tr)

avg_THI_tr=new_train['THI'].mean()          #將THI Nan部分以算術平均數之THI取代
new_train['THI']= new_train['THI'].fillna(avg_THI_tr)

new_train.dropna(subset=['FeedDay','DadID','MomID'], inplace=True)
#new_train

In [ ]:
#處理new_test的
#父親精液編號
new_test['DadID']=new_test['DadID'].fillna("NoData").astype("str")

#將MomID中沒有編號的打為NoData 
new_test['MomID']=new_test['MomID'].fillna("NoData").astype("str")

avg_Temperature=new_test['Temperature'].mean() #將平均溫度Nan部分以算術平均數之平均溫度取代
new_test['Temperature']= new_test['Temperature'].fillna(avg_Temperature)


avg_T_Max=new_test['T Max'].mean()          #將最高溫度Nan部分以算術平均數之最高溫度取代
new_test['T Max']= new_test['T Max'].fillna(avg_T_Max)


avg_RH=new_test['RH'].mean()                  #將相對濕度Nan部分以算術平均數之相對濕度取代
new_test['RH']= new_test['RH'].fillna(avg_RH)


avg_RH_Min=new_test['RHMin'].mean()         #將最小相對濕度Nan部分以算術平均數之最小相對濕度取代
new_test['RHMin']= new_test['RHMin'].fillna(avg_RH_Min)


avg_Precp=new_test['Precp'].mean()          #將降水量Nan部分以算術平均數之降水量取代
new_test['Precp']= new_test['Precp'].fillna(avg_Precp)

avg_THI=new_test['THI'].mean()          #將THI Nan部分以算術平均數之THI取代
new_test['THI']= new_test['THI'].fillna(avg_THI)
#
#new_test


In [ ]:
from datetime import datetime
for i in range(len(new_train['Sampling date'])):
          sdate=datetime.strptime(new_train['Sampling date'][i], "%Y/%m/%d")
          new_train['Sampling D Year'][i]= sdate.year
          new_train['Sampling D Month'][i]= sdate.month
#new_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
for i in range(len(new_test['Sampling date'])):
          sdate=datetime.strptime(new_test['Sampling date'][i], "%Y/%m/%d")
          new_test['Sampling D Year'][i]= sdate.year
          new_test['Sampling D Month'][i]= sdate.month
#new_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
#new_train把當月有病的標記起來(為1)
for i in range(len(spec)):
    dd=datetime.strptime(spec['4'][i], "%Y/%m/%d %H:%M")
    if len(new_train.index[new_train['CowID'] == spec['1'][i]])>0:
        for j in new_train.index[new_train['CowID'] == spec['1'][i]]:
            if new_train['Sampling D Year'][j] == dd.year and new_train['Sampling D Month'][j] == dd.month and new_train['firm'][j]==spec['7'][i]:
                new_train['disease'][j] = 1
                #print(j,new_train['CowID'][j],new_train['Sampling D Year'][j],new_train['Sampling D Month'][j])

#new_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
#new_test把當月有病的標記起來(為1)
for i in range(len(spec)):
    dd=datetime.strptime(spec['4'][i], "%Y/%m/%d %H:%M")
    if len(new_test.index[new_test['CowID'] == spec['1'][i]])>0:
        for j in new_test.index[new_test['CowID'] == spec['1'][i]]:
            if new_test['Sampling D Year'][j] == dd.year and new_test['Sampling D Month'][j] == dd.month and new_test['firm'][j]==spec['7'][i]:
                new_test['disease'][j] = 1
                #print(j,new_test['CowID'][j],new_test['Sampling D Year'][j],new_test['Sampling D Month'][j])

#new_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
new_train=new_train.fillna(0)
new_test=new_test.fillna(0)
new_train.reset_index(inplace=True,drop=True)

In [ ]:
#new_train

In [ ]:
#改new_train
i=0
for i in range(len(new_train['Sampling D Month'])):
          if int(new_train['Sampling D Month'][i]) >=3 and int(new_train['Sampling D Month'][i])<=5:
              new_train['Sampling D Month'][i] = "Spring"
          elif int(new_train['Sampling D Month'][i]) >=6 and int(new_train['Sampling D Month'][i])<=8:
              new_train['Sampling D Month'][i] = "Summer"
          elif int(new_train['Sampling D Month'][i]) >=9 and int(new_train['Sampling D Month'][i])<=11:
              new_train['Sampling D Month'][i] = "Autumn"
          else:
              new_train['Sampling D Month'][i] = "Winter"   
#改new_train的feedday以及babynum
for i in range(len(new_train['ID'])):
    if int(new_train['FeedDay'][i]) <=100:
        new_train['cycle'][i] = 0
    elif int(new_train['FeedDay'][i]) >=101 and int(new_train['FeedDay'][i]) <=200 :
        new_train['cycle'][i] = 1
    elif int(new_train['FeedDay'][i]) >=201 and int(new_train['FeedDay'][i]) <=305:
        new_train['cycle'][i] = 2
    else:
        new_train['cycle'][i] = 3

    if int(new_train['BabyNum'][i]) <=2:
        new_train['tcalving number'][i] = 0
    elif int(new_train['BabyNum'][i])>=7:
        new_train['tcalving number'][i] = 2
    else:
        new_train['tcalving number'][i] = 1

#new_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykerne

In [ ]:
#改new_test
i=0
for i in range(len(new_test['Sampling D Month'])):
          if int(new_test['Sampling D Month'][i]) >=3 and int(new_test['Sampling D Month'][i])<=5:
              new_test['Sampling D Month'][i] = "Spring"
          elif int(new_test['Sampling D Month'][i]) >=6 and int(new_test['Sampling D Month'][i])<=8:
              new_test['Sampling D Month'][i] = "Summer"
          elif int(new_test['Sampling D Month'][i]) >=9 and int(new_test['Sampling D Month'][i])<=11:
              new_test['Sampling D Month'][i] = "Autumn"
          else:
              new_test['Sampling D Month'][i] = "Winter"
#改new_test的feedday以及babynum
for i in range(len(new_test['ID'])):
    if int(new_test['FeedDay'][i]) <=100:
        new_test['cycle'][i] = 0
    elif int(new_train['FeedDay'][i]) >=101 and int(new_test['FeedDay'][i]) <=200 :
        new_test['cycle'][i] = 1
    elif int(new_test['FeedDay'][i]) >=201 and int(new_test['FeedDay'][i]) <=305:
        new_test['cycle'][i] = 2
    else:
        new_test['cycle'][i] = 2
    if int(new_test['BabyNum'][i]) <=2:
        new_test['tcalving number'][i] = 0
    elif int(new_test['BabyNum'][i])>=7:
        new_test['tcalving number'][i] = 2
    else:
        new_test['tcalving number'][i] = 1    
#new_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impor

In [ ]:
new_train=new_train.fillna(0)
new_test=new_test.fillna(0)

In [ ]:
##測試為毛y_test會有0的部分 ，勿管
new_train_backup01 = new_train
new_train_backup01
y__tt=pd.DataFrame(data=new_train_backup01,columns=['ID','Milk'])
#y__tt
#print(y__tt.loc[[3427]])

In [ ]:
y_fortest=new_test['Milk']
y_fortest

0       40.0
1       28.0
2       12.0
3       35.0
4       28.8
        ... 
6678    39.0
6679    10.0
6680    30.6
6681    20.0
6682    16.0
Name: Milk, Length: 6683, dtype: float64

In [ ]:
y=new_train['Milk']
y

0        20.0
1        43.0
2        24.0
3        18.2
4        10.0
         ... 
26475    33.0
26476    24.0
26477    20.4
26478    19.0
26479    25.5
Name: Milk, Length: 26480, dtype: float64

In [ ]:
savefornew_train01=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test01=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train02=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test02=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train03=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test03=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train04=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test04=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train05=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test05=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train06=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test06=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train07=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test07=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train08=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test08=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train09=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test09=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

savefornew_train10=pd.DataFrame(data=new_train,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])
savefornew_test10=pd.DataFrame(data=new_test,columns=['ID','firm','CowID','MomID','BabyNum','FeedDay','Milk','preMilk','Sampling date','Age','BreedNum','Temperature','T Max','RH','RHMin','Precp','THI','cycle','tcalving number','disease','Sampling D Year','Sampling D Month'])

In [ ]:
new_train_backup = new_train
new_test_backup = new_test

In [ ]:
#挑選靠賽要訓練的資料
new_train=pd.DataFrame(data=new_train_backup,columns=['firm','CowID','BabyNum','FeedDay','Age','BreedNum','T Max','THI','disease','Sampling D Month'])
#new_train

In [ ]:
new_test=pd.DataFrame(data=new_test_backup,columns=['firm','CowID','BabyNum','FeedDay','Age','BreedNum','T Max','THI','disease','Sampling D Month'])
#new_test

In [ ]:
#把要one hot的類別轉換成數字
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
all_data=pd.concat([new_train,new_test])
#all_data['3'] = labelencoder.fit_transform(all_data['3'])
all_data['CowID'] = labelencoder.fit_transform(all_data['CowID'])
#all_data['MomID'] = labelencoder.fit_transform(all_data['MomID'])
#all_data['DadID'] = labelencoder.fit_transform(all_data['DadID'])
all_data['Sampling D Month'] = labelencoder.fit_transform(all_data['Sampling D Month'])
new_train = all_data[0:len(new_train)]
new_test = all_data[len(new_train)::]
all_data=pd.concat([new_train,new_test])


In [ ]:
all_data

,firm,CowID,BabyNum,FeedDay,Age,BreedNum,T Max,THI,disease,Sampling D Month
0,2,728,1,55,30,2,27.826169,22.850519,1.0,2
1,2,736,2,62,38,1,34.900000,27.093337,0.0,1
2,2,781,1,11,27,0,23.800000,21.498437,0.0,0
3,1,388,2,290,57,7,15.700000,12.100133,0.0,1
4,2,189,2,156,55,1,20.200000,17.597478,0.0,3
...,...,...,...,...,...,...,...,...,...,...
6678,2,280,3,18,42,0,34.900000,29.323086,0.0,2
6679,2,1790,2,508,52,4,34.800000,28.814617,0.0,2
6680,2,1555,3,158,62,1,34.400000,27.200232,0.0,1
6681,2,466,1,9,22,0,19.500000,16.578618,1.0,3


In [ ]:
#把要的類別轉換成one hot
import numpy as np
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(categorical_features =[0,1,2,5,8,9])#6
enc.fit(all_data)
X=enc.transform(new_train).toarray()
X_test=enc.transform(new_test).toarray()
X.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


(26480, 1858)

In [ ]:
X_test.shape

(6683, 1858)

In [ ]:
new_test

,firm,CowID,BabyNum,FeedDay,Age,BreedNum,T Max,THI,disease,Sampling D Month
0,2,96,3,183,70,2,25.500000,20.236278,0.0,3
1,2,494,3,71,50,0,35.200000,22.838761,0.0,2
2,3,1817,1,365,38,5,27.808703,22.838761,0.0,2
3,2,76,4,177,73,1,29.100000,23.221440,0.0,1
4,1,250,1,42,28,0,11.100000,10.980262,0.0,3
...,...,...,...,...,...,...,...,...,...,...
6678,2,280,3,18,42,0,34.900000,29.323086,0.0,2
6679,2,1790,2,508,52,4,34.800000,28.814617,0.0,2
6680,2,1555,3,158,62,1,34.400000,27.200232,0.0,1
6681,2,466,1,9,22,0,19.500000,16.578618,1.0,3


In [ ]:
X

array([[ 0.        ,  1.        ,  0.        , ..., 30.        ,
        27.82616918, 22.85051932],
       [ 0.        ,  1.        ,  0.        , ..., 38.        ,
        34.9       , 27.09333711],
       [ 0.        ,  1.        ,  0.        , ..., 27.        ,
        23.8       , 21.4984371 ],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., 26.        ,
        16.2       , 13.40103018],
       [ 1.        ,  0.        ,  0.        , ..., 34.        ,
        34.1       , 26.27994148],
       [ 1.        ,  0.        ,  0.        , ..., 70.        ,
        33.8       , 26.86139404]])

In [ ]:
#X=X.values
X = X.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#y=y.values

In [ ]:
X

array([[0.        , 1.        , 0.        , ..., 0.18867923, 0.61618984,
        0.6523535 ],
       [0.        , 1.        , 0.        , ..., 0.2389937 , 0.8338462 ,
        0.82666194],
       [0.        , 1.        , 0.        , ..., 0.16981131, 0.49230766,
        0.5968058 ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.163522  , 0.25846156,
        0.26413873],
       [1.        , 0.        , 0.        , ..., 0.21383648, 0.8092307 ,
        0.7932451 ],
       [1.        , 0.        , 0.        , ..., 0.44025156, 0.79999995,
        0.81713307]], dtype=float32)

In [ ]:
#X_test=X_test.values
X_test=X_test.astype('float32')
X_test=scaler.transform(X_test)

In [ ]:
X_test.shape

(6683, 1858)

In [ ]:
X_test= np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [ ]:
X_test.shape

(6683, 1, 1858)

In [ ]:
from keras import backend as K
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
from sklearn.model_selection import KFold
from keras import regularizers
from matplotlib import pyplot
from keras import optimizers
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Embedding, Bidirectional,TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from numpy import ndarray
import numpy as np
import matplotlib.pyplot as plt
time=1
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(X):
    x_train, x_test =X[train_index], X[test_index],
    y_train, y_test = y[train_index], y[test_index]
    print("Please show me the shape of x_train:",x_train.shape)
    print("Please show me the shape of x_test:",x_test.shape)
    print("Please show me the shape of y_train:",y_train.shape)
    print("Please show me the shape of y_test:",y_test.shape)

    print("y_test:") #看每次y_test一不一樣
    print(y_test)

    x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))#做成三維
    x_test = np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))

    model = Sequential()#LSTM
    model.add(LSTM(64, input_shape=(1,x_train.shape[2])))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    ADAM=keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=rmse, optimizer=ADAM,metrics=[rmse])
    early_stopping=EarlyStopping(monitor='val_loss', patience=10)
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    history=model.fit(x_train, y_train, epochs=500,validation_data=(x_test, y_test), batch_size=5, verbose=2, callbacks=[early_stopping])#epochs要改回1000

    Predicts = model.predict(x_train)#TrainRMSE
    testScore = math.sqrt(mean_squared_error(y_train, Predicts))
    print('Train Score: %.2f RMSE' % (testScore))
    

    Predict = model.predict(x_test)#TestRMSE
    testScore = math.sqrt(mean_squared_error(y_test, Predict))
    print('Validation Score: %.2f RMSE' % (testScore))
    #print(Predict)

    Prediction = model.predict(X_test)
    Prediction
    testScore = math.sqrt(mean_squared_error(y_fortest, Prediction))
    print('Test Score: %.2f RMSE' % (testScore))

    if time == 1:
      i=0
      for i in range(len(y_train)):
        savefornew_train01['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train01['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test01['preMilk'][y_fortest.index[i]]=Prediction[i]
      
      print(savefornew_train01)

    elif time == 2:
      i=0
      for i in range(len(y_train)):
        savefornew_train02['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train02['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test02['preMilk'][y_fortest.index[i]]=Prediction[i]

      print(savefornew_train02)

    elif time == 3:
      i=0
      for i in range(len(y_train)):
        savefornew_train03['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train03['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test03['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 4:
      i=0
      for i in range(len(y_train)):
        savefornew_train04['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train04['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test04['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 5:
      i=0
      for i in range(len(y_train)):
        savefornew_train05['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train05['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test05['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 6:
      i=0
      for i in range(len(y_train)):
        savefornew_train06['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train06['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test06['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 7:
      i=0
      for i in range(len(y_train)):
        savefornew_train07['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train07['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test07['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 8:
      i=0
      for i in range(len(y_train)):
        savefornew_train08['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train08['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test08['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 9:
      i=0
      for i in range(len(y_train)):
        savefornew_train09['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train09['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test09['preMilk'][y_fortest.index[i]]=Prediction[i]
    elif time == 7:
      i=0
      for i in range(len(y_train)):
        savefornew_train10['preMilk'][y_train.index[i]]=Predicts[i]
      i=0
      for i in range(len(y_test)):
        savefornew_train10['preMilk'][y_test.index[i]]=Predict[i]
      i=0
      for i in range(len(y_fortest)):
        savefornew_test10['preMilk'][y_fortest.index[i]]=Prediction[i]
    time=time+1

Please show me the shape of x_train: (23832, 1858)
Please show me the shape of x_test: (2648, 1858)
Please show me the shape of y_train: (23832,)
Please show me the shape of y_test: (2648,)
y_test:
6        15.4
17       25.0
34       23.0
35       17.0
52       28.0
         ... 
26388    19.9
26428    34.0
26436    17.0
26437    16.0
26448    17.2
Name: Milk, Length: 2648, dtype: float64
Epoch 1/500
4767/4767 - 22s - loss: 8.9134 - rmse: 8.9137 - val_loss: 6.5702 - val_rmse: 6.5701
Epoch 2/500
4767/4767 - 19s - loss: 6.9605 - rmse: 6.9603 - val_loss: 6.0142 - val_rmse: 6.0146
Epoch 3/500
4767/4767 - 19s - loss: 6.4275 - rmse: 6.4279 - val_loss: 5.5945 - val_rmse: 5.5946
Epoch 4/500
4767/4767 - 19s - loss: 6.1016 - rmse: 6.1015 - val_loss: 5.3876 - val_rmse: 5.3874
Epoch 5/500
4767/4767 - 21s - loss: 5.9076 - rmse: 5.9070 - val_loss: 5.2481 - val_rmse: 5.2489
Epoch 6/500
4767/4767 - 19s - loss: 5.7997 - rmse: 5.7997 - val_loss: 5.2323 - val_rmse: 5.2333
Epoch 7/500
4767/4767 - 19s - l

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


          ID  firm     CowID  ... disease  Sampling D Year  Sampling D Month
0      10806     2   3126105  ...     1.0           2016.0            Summer
1      10950     2   3126118  ...     0.0           2017.0            Spring
2      30007     2   3126179  ...     0.0           2016.0            Autumn
3       2137     1   2052011  ...     0.0           2018.0            Spring
4      20955     2    127993  ...     0.0           2016.0            Winter
...      ...   ...       ...  ...     ...              ...               ...
26475  15392     2   2122400  ...     0.0           2018.0            Spring
26476  25883     2  10837415  ...     0.0           2018.0            Autumn
26477  19848     2    124308  ...     0.0           2013.0            Winter
26478   3360     1   1051856  ...     0.0           2015.0            Summer
26479   6626     1  96051858  ...     0.0           2013.0            Summer

[26480 rows x 22 columns]
Please show me the shape of x_train: (23832, 1858

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


          ID  firm     CowID  ... disease  Sampling D Year  Sampling D Month
0      10806     2   3126105  ...     1.0           2016.0            Summer
1      10950     2   3126118  ...     0.0           2017.0            Spring
2      30007     2   3126179  ...     0.0           2016.0            Autumn
3       2137     1   2052011  ...     0.0           2018.0            Spring
4      20955     2    127993  ...     0.0           2016.0            Winter
...      ...   ...       ...  ...     ...              ...               ...
26475  15392     2   2122400  ...     0.0           2018.0            Spring
26476  25883     2  10837415  ...     0.0           2018.0            Autumn
26477  19848     2    124308  ...     0.0           2013.0            Winter
26478   3360     1   1051856  ...     0.0           2015.0            Summer
26479   6626     1  96051858  ...     0.0           2013.0            Summer

[26480 rows x 22 columns]
Please show me the shape of x_train: (23832, 1858

...

Please show me the shape of x_train: (23832, 1858)
Please show me the shape of x_test: (2648, 1858)
Please show me the shape of y_train: (23832,)
Please show me the shape of y_test: (2648,)
y_test:
1        43.0
18       46.0
48       36.1
66       33.9
74       18.0
         ... 
26412    14.0
26416    27.8
26426    16.3
26446    13.3
26453    18.0
Name: Milk, Length: 2648, dtype: float64
Epoch 1/500
4767/4767 - 24s - loss: 8.8920 - rmse: 8.8912 - val_loss: 6.5140 - val_rmse: 6.5128
Epoch 2/500
4767/4767 - 20s - loss: 6.9689 - rmse: 6.9684 - val_loss: 6.0579 - val_rmse: 6.0560
Epoch 3/500
4767/4767 - 20s - loss: 6.4432 - rmse: 6.4431 - val_loss: 5.5425 - val_rmse: 5.5402
Epoch 4/500
4767/4767 - 20s - loss: 6.1311 - rmse: 6.1318 - val_loss: 5.3036 - val_rmse: 5.3011
Epoch 5/500
4767/4767 - 20s - loss: 5.9527 - rmse: 5.9524 - val_loss: 5.1183 - val_rmse: 5.1159
Epoch 6/500
4767/4767 - 20s - loss: 5.8311 - rmse: 5.8318 - val_loss: 5.0541 - val_rmse: 5.0517
Epoch 7/500
4767/4767 - 23s - l

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Please show me the shape of x_train: (23832, 1858)
Please show me the shape of x_test: (2648, 1858)
Please show me the shape of y_train: (23832,)
Please show me the shape of y_test: (2648,)
y_test:
16       25.0
24       34.0
25       13.0
40       26.0
53       36.6
         ... 
26430    28.0
26439    34.0
26457    42.0
26469    32.0
26475    33.0
Name: Milk, Length: 2648, dtype: float64
Epoch 1/500
4767/4767 - 20s - loss: 8.9219 - rmse: 8.9209 - val_loss: 6.7324 - val_rmse: 6.7338
Epoch 2/500
4767/4767 - 20s - loss: 6.9649 - rmse: 6.9652 - val_loss: 6.1704 - val_rmse: 6.1712
Epoch 3/500
4767/4767 - 21s - loss: 6.4358 - rmse: 6.4370 - val_loss: 5.7413 - val_rmse: 5.7414
Epoch 4/500
4767/4767 - 20s - loss: 6.1130 - rmse: 6.1129 - val_loss: 5.4893 - val_rmse: 5.4896
Epoch 5/500
4767/4767 - 20s - loss: 5.9098 - rmse: 5.9091 - val_loss: 5.4210 - val_rmse: 5.4213
Epoch 6/500
4767/4767 - 21s - loss: 5.7992 - rmse: 5.7989 - val_loss: 5.2735 - val_rmse: 5.2740
Epoch 7/500
4767/4767 - 22s - l

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Please show me the shape of x_train: (23832, 1858)
Please show me the shape of x_test: (2648, 1858)
Please show me the shape of y_train: (23832,)
Please show me the shape of y_test: (2648,)
y_test:
9        28.0
11       21.0
13       30.0
55        6.0
60       37.0
         ... 
26460    27.6
26461    36.0
26462    14.0
26476    24.0
26478    19.0
Name: Milk, Length: 2648, dtype: float64
Epoch 1/500
4767/4767 - 20s - loss: 8.9178 - rmse: 8.9178 - val_loss: 6.8404 - val_rmse: 6.8390
Epoch 2/500
4767/4767 - 20s - loss: 6.9712 - rmse: 6.9709 - val_loss: 6.3569 - val_rmse: 6.3563
Epoch 3/500
4767/4767 - 19s - loss: 6.4397 - rmse: 6.4396 - val_loss: 6.0245 - val_rmse: 6.0234
Epoch 4/500
4767/4767 - 19s - loss: 6.0667 - rmse: 6.0675 - val_loss: 5.7727 - val_rmse: 5.7719
Epoch 5/500
4767/4767 - 20s - loss: 5.9263 - rmse: 5.9261 - val_loss: 5.5865 - val_rmse: 5.5860
Epoch 6/500
4767/4767 - 19s - loss: 5.8199 - rmse: 5.8197 - val_loss: 5.5552 - val_rmse: 5.5539
Epoch 7/500
4767/4767 - 19s - l

In [ ]:
savefornew_train01

,ID,firm,CowID,MomID,BabyNum,FeedDay,Milk,preMilk,Sampling date,Age,BreedNum,Temperature,T Max,RH,RHMin,Precp,THI,cycle,tcalving number,disease,Sampling D Year,Sampling D Month
0,10806,2,3126105,116686.0,1,55,20.0,19.802689,2016/7/26,30,2,23.967792,27.826169,78.502478,59.891615,0.0,22.850519,0.0,0.0,1.0,2016.0,Summer
1,10950,2,3126118,116438.0,2,62,43.0,40.369633,2017/4/17,38,1,28.800000,34.900000,79.000000,51.000000,0.0,27.093337,0.0,0.0,0.0,2017.0,Spring
2,30007,2,3126179,99116633.0,1,11,24.0,27.622871,2016/11/23,27,0,21.800000,23.800000,93.000000,85.000000,24.5,21.498437,0.0,0.0,0.0,2016.0,Autumn
3,2137,1,2052011,41323.0,2,290,18.2,19.157009,2018/3/8,57,7,12.000000,15.700000,91.000000,68.000000,26.5,12.100133,2.0,0.0,0.0,2018.0,Spring
4,20955,2,127993,98111275.0,2,156,10.0,14.273954,2016/2/22,55,1,17.700000,20.200000,95.000000,86.000000,0.0,17.597478,1.0,0.0,0.0,2016.0,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26475,15392,2,2122400,99116273.0,3,137,33.0,32.756561,2018/5/19,60,1,28.500000,31.900000,83.000000,66.000000,0.0,27.146230,1.0,1.0,0.0,2018.0,Spring
26476,25883,2,10837415,99999999.0,1,216,24.0,23.847013,2018/10/23,33,1,24.600000,27.900000,91.000000,67.000000,0.0,24.074355,2.0,0.0,0.0,2018.0,Autumn
26477,19848,2,124308,95110546.0,1,37,20.4,22.016079,2013/12/29,26,0,13.300000,16.200000,74.000000,61.000000,0.0,13.401030,0.0,0.0,0.0,2013.0,Winter
26478,3360,1,1051856,96040548.0,1,73,19.0,20.405935,2015/8/10,34,0,28.800000,34.100000,69.000000,38.000000,0.0,26.279941,0.0,0.0,0.0,2015.0,Summer


In [ ]:
savefornew_train02

,ID,firm,CowID,MomID,BabyNum,FeedDay,Milk,preMilk,Sampling date,Age,BreedNum,Temperature,T Max,RH,RHMin,Precp,THI,cycle,tcalving number,disease,Sampling D Year,Sampling D Month
0,10806,2,3126105,116686.0,1,55,20.0,22.278074,2016/7/26,30,2,23.967792,27.826169,78.502478,59.891615,0.0,22.850519,0.0,0.0,1.0,2016.0,Summer
1,10950,2,3126118,116438.0,2,62,43.0,41.734718,2017/4/17,38,1,28.800000,34.900000,79.000000,51.000000,0.0,27.093337,0.0,0.0,0.0,2017.0,Spring
2,30007,2,3126179,99116633.0,1,11,24.0,28.850533,2016/11/23,27,0,21.800000,23.800000,93.000000,85.000000,24.5,21.498437,0.0,0.0,0.0,2016.0,Autumn
3,2137,1,2052011,41323.0,2,290,18.2,18.646597,2018/3/8,57,7,12.000000,15.700000,91.000000,68.000000,26.5,12.100133,2.0,0.0,0.0,2018.0,Spring
4,20955,2,127993,98111275.0,2,156,10.0,15.523809,2016/2/22,55,1,17.700000,20.200000,95.000000,86.000000,0.0,17.597478,1.0,0.0,0.0,2016.0,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26475,15392,2,2122400,99116273.0,3,137,33.0,32.306976,2018/5/19,60,1,28.500000,31.900000,83.000000,66.000000,0.0,27.146230,1.0,1.0,0.0,2018.0,Spring
26476,25883,2,10837415,99999999.0,1,216,24.0,24.393471,2018/10/23,33,1,24.600000,27.900000,91.000000,67.000000,0.0,24.074355,2.0,0.0,0.0,2018.0,Autumn
26477,19848,2,124308,95110546.0,1,37,20.4,23.213276,2013/12/29,26,0,13.300000,16.200000,74.000000,61.000000,0.0,13.401030,0.0,0.0,0.0,2013.0,Winter
26478,3360,1,1051856,96040548.0,1,73,19.0,21.881302,2015/8/10,34,0,28.800000,34.100000,69.000000,38.000000,0.0,26.279941,0.0,0.0,0.0,2015.0,Summer


In [ ]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 64)                492288    
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 65        
Total params: 492,353
Trainable params: 492,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pandas as pd
#存下做完預處理的train存下做完預處理的train
savefornew_train09.to_csv("savefornew_train09.csv",index=False,sep=',')
files.download('savefornew_train09.csv')
#字典中的key值即為csv中列名
savefornew_test09.to_csv("savefornew_test09.csv",index=False,sep=',')
files.download('savefornew_train09.csv')